# Solving problems by Searching

This notebook serves as supporting material for topics covered in **Chapter 3 - Solving Problems by Searching** and **Chapter 4 - Beyond Classical Search** from the book *Artificial Intelligence: A Modern Approach.* This notebook uses implementations from [search.py](https://github.com/aimacode/aima-python/blob/master/search.py) module. Let's start by importing everything from search module.

In [85]:

import random
import bisect
# Needed to hide warnings in the matplotlib sections
import warnings
import pandas as pd
import time
import os
import statistics
#from numpy import *
warnings.filterwarnings("ignore")

For visualisations, we use networkx and matplotlib to show the map in the notebook and we use ipywidgets to interact with the map to see how the searching algorithm works. These are imported as required in `notebook.py`.

## CONTENTS

* Genetic Algorithm


## GENETIC ALGORITHM FOR TSP

Genetic algorithms (or GA) are inspired by natural evolution and are particularly useful in optimization and search problems with large state spaces.

Given a problem, algorithms in the domain make use of a *population* of solutions (also called *states*), where each solution/state represents a feasible solution. At each iteration (often called *generation*), the population gets updated using methods inspired by biology and evolution, like *crossover*, *mutation* and *natural selection*.

PERMUTATION ENCODING

GA Parameters:
We now need to define the maximum size of each population. Larger populations have more variation but are computationally more  expensive to run algorithms on.
As our population is not very large, we can afford to keep a relatively large mutation rate.
Termination after a predefined number of generations.
N is the size of the chromosmes, and [0,..,N-1] is the alphabet

Great! Now, we need to define the most important metric for the genetic algorithm, i.e the fitness function. This will simply return the number of matching characters between the generated sample and the target phrase.

In [86]:
class Grafo:
    """Un Grafo es un array bidimensional de tamño N*N, siendo N el número de ciudades,
    representadas por los valores 0,...,N-1; 0 se toma como ciudad de partida.
    El valor de la posición (x,y) es la distancia entre las ciudades x e y"""
    def __init__(self,lista):
        """N es el número de ciudades correspondiente a la lista de valores de una instancia en la sintaxis 
        EDGE_WEIGHT_TYPE"""
        self.N = int(((8*len(lista)+1)**0.5)-1)/2
        self.ciudades = list(range(int(self.N)))
        self.Dist = [[0]*int(self.N) for i in self.ciudades]
        x_acc = 0
        for x in range(int(self.N)):
            x_acc += x
            for y in range(x+1):
                self.Dist[x][y] = self.Dist[y][x] = lista[x_acc+y]

In [87]:
def fitness_fn_initial(sample):
    length = min(len(sample), len(target))
    fitness = sum(1 for i in range(length) if sample[i] == target[i])
    return fitness

# target
target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

def total_distance(sample):
    total = 0
    for i in range(len(sample) - 1):
        total += grafo.Dist[sample[i]][sample[i + 1]]
    # Añadir la distancia para volver a la ciudad de inicio
    total += grafo.Dist[sample[-1]][sample[0]]
    return total

def fitness_fn(sample):
    total = total_distance(sample)
    return 1 / total if total > 0 else float('inf')

To generate `ngen` number of generations, we run a `for` loop `ngen` number of times. After each generation, we calculate the fitness of the best individual of the generation and compare it to the value of `f_thres` using the `fitness_threshold` function. After every generation, we print out the best individual of the generation and the corresponding fitness value. Lets now write a function to do this.

In [88]:
def genetic_algorithm_stepwise(population, fitness_fn, gene_pool, ngen=1200, pmut=0.05):
    for generation in range(int(ngen)):
        # Dictionary to store the results of the current generation
        result ={"maxFitness":[], "avgFitness":[],"maxCost":[], "avgCost":[]}
        
        # Elitism
        previous_best = max(population, key=fitness_fn)
        # Apply genetic operators (crossover and mutation)
        population = [mutate(uniform_crossover(*select(2, population, fitness_fn)), pmut) for i in range(len(population)-1)]
        # Add the best individual from the previous generation
        population.append(previous_best)
        # Find the current best individual
        current_best = max(population, key=fitness_fn)
        # Fitness values for the current population
        fitness_values = list(map(fitness_fn, population))
        # Costs associated with current population
        costs = list(map(total_distance, population))
        
        # Store values in dictionary
        maxFitness = max(fitness_values)
        result["maxFitness"].append(maxFitness)
        # The mean in statistics is the average of a particular sample or data collection
        avgFitness = statistics.mean(fitness_values)
        result["avgFitness"].append(avgFitness)
        maxCost = max(costs)
        result["maxCost"].append(maxCost)
        avgCost = statistics.mean(costs)
        result["avgCost"].append(avgCost)
        
        print(f'Current best: {current_best}\t\tGeneration: {generation}\t\tMaxFitness: {maxFitness}\t\tAvgFitness: {avgFitness}\t\tMaxCost:{maxCost}\t\tAvgCost: {avgFitness}\r')
    return max(population, key=fitness_fn)

def init_population(pop_number, gene_pool):
    # a chromosome is a random permutation of the alphabet
    population = []
    for _ in range(pop_number):
        # Shuffle the gene pool and take the first pool_size elements as an individual
        v = gene_pool[:]
        random.shuffle(v)
        population.append(v)
    return population

def select(r, population, fitness_fn):
    fitnesses = map(fitness_fn, population)
    #scaling here
    sampler = weighted_sampler(population, fitnesses)
    return [sampler() for i in range(r)]

def weighted_sampler(seq, weights):
    """Return a random-sample function that picks from seq weighted by weights."""
    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)
    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]
    # bisect(a,x) -> insertion position of a in a sorted list x - AL REVES

def uniform_crossover(x, y):
    # x, y permutations of the alphabet
    n = 0
    child = [-1] * (N-1)
    indexes = [0] * (N-1)
    # de x se copian los valores de las posiciones con indexex[i] == 1 en las mismas posiciones en child
    for i in range(N-1):
        indexes[i] = random.randint(0, 1) 
        if indexes[i] == 1:
            child[i] = x[i]
            n += 1
    # El resto (N-n) se copia de y en su orden relativo, desde el principio
    i = 0  # índice en y
    k = 0  # índice en child
    for t in range(N - 1 - n):
        while y[i] in child[:]:
            i += 1
        while child[k] != -1:
            k += 1
        child[k] = y[i]
        i += 1
    return child

def order_crossover(parent1, parent2):
    start = random.randint(0, len(parent1) - 2)
    end = random.randint(start + 1, len(parent1) - 1)
    child_p1 = parent1[start:end]
    child = child_p1 + [city for city in parent2 if city not in child_p1]
    return child

def mutate(x, pmut):    
    for swapped in range(len(x)):
        if random.random() < pmut:
            swap_with = random.randint(0, len(x) - 1)
            x[swapped], x[swap_with] = x[swap_with], x[swapped]
    return x

The function defined above is essentially the same as the one defined in `search.py` with the added functionality of printing out the data of each generation.

We have defined all the required functions and variables. Let's now create a new population and test the function we wrote above.

In [89]:
prueba = [ 
    0,
    1, 0,
    6, 2, 0,
    4, 5, 3, 0,
]

gr21 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
    240, 290, 590, 140, 480, 255, 205, 220, 515, 150, 100, 170, 390, 425, 255, 395, 205, 220, 155, 0, 
    380, 140, 495, 280, 480, 340, 350, 370, 505, 185, 240, 310, 345, 280, 105, 380, 280, 165, 305, 150, 0,
]

gr17 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
    70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 
    211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 
    268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0, 
    246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, 0, 
    121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, 0,
 ]

gr12 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
 ]

target = [15, 11, 8, 4, 1, 9, 10, 2, 14, 13, 16, 5, 7, 6, 12, 3] # used for basic fitness function
print(target)

[15, 11, 8, 4, 1, 9, 10, 2, 14, 13, 16, 5, 7, 6, 12, 3]


In [90]:
# Guarda un archivo Excel con los datos obtenidos de la ejecución en la hoja pasada como parámetro
def saveExcelFile(t, target, solution, fitness, cost, sheet, chromosomeSize):
    output_file_path = "search3e-GeneticAlgorithm.xlsx"    
    df = pd.DataFrame([[t, target, solution, fitness, cost, chromosomeSize]], 
                      columns=['Time', 'Target', 'Solution', 'Fitness', 'Cost','Chromosome Size'])    
    if not os.path.exists(output_file_path):
        # Si el archivo no existe ya, lo crea y añade los datos
        with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name=sheet, index=False)
    else:
        # En otro caso, añade los datos al fichero existente
        with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            try:
                # Lee los datos ya existentes en la hoja del fichero
                existing_df = pd.read_excel(output_file_path, sheet_name=sheet)
                df = pd.concat([existing_df, df], ignore_index=True)
            except ValueError:
                # Si no existe la hoja en el fichero la crea
                pass
            
            df.to_excel(writer, sheet_name=sheet, index=False)
      

In [93]:
# Población inicial
max_population = 200
# Creo el grafo
#grafo = Grafo(gr12)
#grafo = Grafo(gr17)
grafo = Grafo(gr21)

# N es el tamaño del cromosoma
N = int(grafo.N)
gene_pool = list(range(1, N))
population = init_population(max_population, gene_pool)
# print(population)

# El 10% de los cromosomas mutan
mutation_rate = 0.1
# Máximo de generaciones
ngen = 200

# Imprimo resultados y guardo datos en Excel
for i in range(10):
    initial_time = time.time()
    solution = genetic_algorithm_stepwise(population, fitness_fn, gene_pool, ngen, mutation_rate)
    end_time = time.time() - initial_time

    print("Time Taken: ", end_time)
    print("Target: ", target)
    print("Solution: ", solution)
    fitness = fitness_fn(solution)
    print("Fitness: ", fitness)
    cost = total_distance(solution)
    print("Cost:", cost)

    saveExcelFile(end_time, target, solution, fitness, cost, 'ox', N)


Current best: [5, 11, 3, 2, 17, 9, 7, 14, 4, 8, 15, 10, 12, 13, 1, 20, 19, 6, 16, 18]		Generation: 0		MaxFitness: 0.00017962996227770793		AvgFitness: 0.00013729617976236444		MaxCost:9035		AvgCost: 0.00013729617976236444
Current best: [10, 11, 7, 5, 9, 12, 13, 1, 6, 15, 18, 19, 20, 16, 14, 17, 3, 8, 4, 2]		Generation: 1		MaxFitness: 0.00018508236165093466		AvgFitness: 0.00013691000347018626		MaxCost:9115		AvgCost: 0.00013691000347018626
Current best: [10, 11, 7, 5, 9, 12, 13, 1, 6, 15, 18, 19, 20, 16, 14, 17, 3, 8, 4, 2]		Generation: 2		MaxFitness: 0.00018508236165093466		AvgFitness: 0.00013672425841672645		MaxCost:8550		AvgCost: 0.00013672425841672645
Current best: [20, 10, 6, 7, 3, 5, 2, 8, 16, 18, 15, 11, 19, 4, 14, 1, 17, 9, 12, 13]		Generation: 3		MaxFitness: 0.0001939487975174554		AvgFitness: 0.00013830274282206862		MaxCost:9080		AvgCost: 0.00013830274282206862
Current best: [5, 3, 9, 19, 16, 18, 11, 7, 6, 4, 1, 14, 20, 13, 2, 8, 17, 12, 10, 15]		Generation: 4		MaxFitness: 0.00019

The genetic algorithm was able to converge!
We implore you to rerun the above cell and play around with `target, max_population, f_thres, ngen` etc parameters to get a better intuition of how the algorithm works. To summarize, if we can define the problem states in simple array format and if we can create a fitness function to gauge how good or bad our approximate solutions are, there is a high chance that we can get a satisfactory solution using a genetic algorithm. 
- There is also a better GUI version of this program `genetic_algorithm_example.py` in the GUI folder for you to play around with.

<br>
This concludes the notebook.
Hope you learned something new!